<a href="https://colab.research.google.com/github/mahaairshad/CE888_si19783/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab5: Recommender**

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [74]:
df = pd.read_csv('https://raw.githubusercontent.com/albanda/CE888/master/lab5-recommender/jester-data-1.csv',header=None)
print(df)

       0      1      2      3      4    ...    96     97     98     99     100
0       74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1      100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2       49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3       48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4       91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60
...    ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
24978  100   0.44   7.43   9.08   2.33  ...   9.03   6.55   8.69   8.79   7.43
24979   91   9.13  -8.16   8.59   9.08  ...  -8.20  -7.23  -8.59   9.13   8.45
24980   39  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24981   37  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
24982   72   2.43   2.67  -3.98   4.27  ...  99.00  99.00  99.00  99.00  99.00

[24983 rows x 101 columns]


In [0]:
joke_count=df[0]
df=df.drop(columns=0)

In [0]:
data=df.values

In [77]:
data.shape

(24983, 100)

# Separate 10% data for validation set by assigning 99 (NaN)

In [0]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(data!=99) #ignoring all the ratings that were never available for the test guys
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_data, idx = replace(data, 0.1)

In [0]:
Actual_data=data.copy() #all the data with the actual rating [rows=users],[columns=items/products]

In [0]:
Train_data=new_data.copy() #10% of values from 'data' set to 99 to be used as test data later [rows=users],[columns=items/products]

In [83]:
Actual_data.shape

(24983, 100)

#Inferring hidden ratings by latent factor modeling

In [0]:
def predict_rating(user_row,item_row):
  user_values=user_preferences[user_row]
  item_values=item_factors[item_row]
  return user_values.dot(item_values)

In [0]:
def train(user_row, item_row, rating, alpha):
  err= rating - predict_rating(user_row, item_row)
  temp= user_preferences[user_row]
  user_preferences[user_row] += alpha * err * item_factors[item_row]
  item_factors[item_row] += alpha * err * temp
  return err

In [0]:
def sgd_svd(iterations,alpha):
  for i in range(iterations):
    error=[]
    for user_row in range (user_preferences.shape[0]):
      for item_row in range (item_factors.shape[0]):
        rating= user_ratings[user_row][item_row]
        if rating != 99:
          err=train(user_row, item_row, rating, alpha)
          error.append(err)
  mse = (np.array(error) ** 2).mean()          
  print(mse)

#Changing hyper-parameters for better results

In [0]:
# Evaluating recommender on test data
def eval_on_ValData():
  error=[]
  pred=user_preferences.dot(item_factors.T)
  error=Actual_data[idx]-pred[idx]
  mse = (np.array(error) ** 2).mean()
  print(mse)

Latent factors=2, Learning rate = 0.0001, No. of SVD iter = 5

In [88]:
latent_factors=2
user_preferences=np.random.rand(len(Train_data[:,0]),latent_factors) #creating random array of same shape as that of Train_data
item_factors=np.random.rand(len(Train_data[0,:]),latent_factors)
user_ratings = Train_data.copy() 
for _ in range(10):
  sgd_svd(5,0.0001)   # reduced no. of iterations to 5 to reduce processing time

24.550659715878115
23.459905228496048
22.374765618134
21.22807216969058
19.69990671221712
18.239598247487425
17.573259923405104
17.325741547639904
17.214513710204223
17.156038787406345


In [115]:
eval_on_ValData() #Test on Validation Data

18.181513066140308


Latent factors=4, Learning rate = 0.0001, No. of SVD iter = 5

In [116]:
latent_factors=4
user_preferences=np.random.rand(len(Train_data[:,0]),latent_factors) #creating random array of same shape as that of Train_data
item_factors=np.random.rand(len(Train_data[0,:]),latent_factors)
for _ in range(10):
  sgd_svd(5,0.0001)  

24.6372665000038
24.037620837198574
23.33996276657925
22.205649385458067
20.015836079514337
18.22466212834178
17.629007364798944
17.408762514589476
17.28439529279141
17.189563803175552


In [117]:
eval_on_ValData()

18.152446119603542


Latent factors=6, Learning rate = 0.0001, No. of SVD iter = 10

In [124]:
latent_factors=6
user_preferences=np.random.rand(len(Train_data[:,0]),latent_factors) #creating random array of same shape as that of Train_data
item_factors=np.random.rand(len(Train_data[0,:]),latent_factors)
for _ in range(10):
  sgd_svd(10,0.0001)  

24.28379285109353
21.992007231090035
17.970908036896823
17.441760541292098
17.133386280559158
16.71002930992531
16.189414352588894
15.728652329394611
15.387871386533345
15.14059080501432


In [125]:
eval_on_ValData()

17.144824666404638


#Visual Comparison

In [0]:
Compar=np.empty(data.shape,dtype=object)
for user_row in range (user_preferences.shape[0]):
    for item_row in range (item_factors.shape[0]):
      rating= Actual_data[user_row][item_row]
      pred_rating=predict_rating(user_row,item_row)
      Compar[user_row][item_row]=str(rating)+"|"+str(np.round(pred_rating,decimals=2))  

In [127]:
import pandas as pd

df = pd.DataFrame(data=Compar)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,-7.82|-4.04,8.79|-4.41,-9.66|-5.74,-8.16|-9.51,-7.52|-2.98,-8.5|-1.61,-9.85|-4.62,4.17|-4.03,-8.98|-8.79,-4.76|-3.93,-8.5|-1.67,-6.75|-1.31,-7.18|-6.33,8.45|-1.4,-7.18|-6.45,-7.52|-9.59,-7.43|-4.46,-9.81|-4.88,-9.85|-3.25,-9.85|-5.61,-9.37|0.37,1.5|-4.23,-4.37|-5.69,-9.81|-10.49,-8.5|-4.52,1.12|-1.47,7.82|3.26,2.86|-1.11,9.13|3.49,-7.43|-7.57,2.14|1.06,-4.08|3.43,-9.08|-9.11,7.82|-2.56,5.05|3.02,4.95|3.55,-9.17|-8.95,-8.4|-3.12,-8.4|-2.26,-8.4|-3.28,...,8.59|1.38,3.59|2.47,-6.84|-4.71,-9.03|-8.17,2.82|1.15,-1.36|1.71,-9.08|-8.86,8.3|2.14,5.68|1.79,-4.81|-4.29,99.0|-6.48,99.0|0.99,99.0|-3.5,99.0|-9.19,99.0|-6.21,99.0|0.37,99.0|-4.04,-9.42|-2.33,99.0|-5.35,99.0|-2.66,99.0|-2.11,-7.72|-4.46,99.0|-0.62,99.0|-4.39,99.0|-4.43,99.0|-6.43,99.0|-1.37,99.0|-1.28,99.0|3.4,99.0|-4.76,2.82|-1.62,99.0|-2.89,99.0|0.46,99.0|-4.75,99.0|-3.76,99.0|-2.67,-5.63|-2.03,99.0|-2.68,99.0|-5.7,99.0|-2.97
1,4.08|4.4,-0.29|5.15,6.36|5.14,4.37|5.64,-2.38|4.04,-9.66|1.4,-0.73|2.92,-5.34|1.96,8.88|4.4,9.22|4.21,6.75|3.16,8.64|1.71,4.42|3.25,7.43|2.19,4.56|3.34,-0.97|2.87,4.66|2.92,-0.68|1.59,3.3|0.86,-1.21|0.73,0.87|2.94,8.64|4.24,8.35|2.94,9.17|5.21,0.05|3.94,7.57|2.52,4.71|0.95,0.87|-0.35,-0.39|1.23,6.99|3.34,6.5|1.74,-0.92|-0.25,7.14|4.74,9.03|2.97,-1.8|0.56,0.73|0.87,7.09|4.85,3.4|2.18,-0.87|2.57,7.91|3.04,...,-6.7|0.06,-3.35|-0.76,-9.03|0.81,4.47|4.49,4.08|2.21,-3.83|0.19,8.74|4.96,1.12|-1.35,0.78|1.05,7.52|2.28,-5.0|1.08,2.77|2.2,8.3|3.22,7.77|4.98,7.33|4.34,6.21|2.23,7.72|2.8,8.98|3.1,8.64|3.64,8.2|2.98,3.93|5.37,4.85|3.01,4.85|3.52,6.07|2.06,8.98|4.33,4.51|4.49,-0.05|3.12,3.69|2.55,4.56|-0.05,0.58|2.15,2.82|3.09,-4.95|2.02,-0.29|1.1,7.86|3.63,-0.19|2.43,-2.14|2.15,3.06|1.99,0.34|3.72,-4.32|2.31,1.07|4.63
2,99.0|6.15,99.0|5.82,99.0|6.36,99.0|5.82,9.03|3.95,9.27|8.88,9.03|4.63,9.27|3.12,99.0|7.34,99.0|6.57,7.33|7.78,7.57|8.98,9.37|2.08,6.17|8.68,-6.36|1.36,-6.89|1.78,-7.86|2.29,9.03|4.58,9.03|6.36,9.03|6.46,7.28|7.43,99.0|7.67,8.25|7.76,99.0|5.7,99.0|7.4,7.48|8.05,7.28|7.43,7.28|9.53,8.93|6.92,99.0|8.26,6.17|7.71,7.28|8.29,99.0|6.5,99.0|7.55,8.98|8.07,7.33|7.1,99.0|5.26,6.17|8.52,9.08|8.06,7.33|8.11,...,6.46|9.13,7.28|9.34,99.0|8.99,99.0|6.48,7.04|7.31,7.28|8.51,99.0|6.34,7.28|9.26,8.25|8.16,99.0|8.23,99.0|6.6,99.0|6.98,99.0|6.86,99.0|4.66,99.0|4.82,8.93|6.48,99.0|6.4,99.0|6.98,99.0|6.19,9.08|7.19,99.0|5.49,99.0|6.75,99.0|6.13,99.0|7.45,99.0|6.63,99.0|5.18,99.0|6.11,99.0|6.64,99.0|7.42,9.03|7.43,99.0|6.13,99.0|7.07,99.0|7.46,9.08|6.4,99.0|6.57,99.0|6.97,99.0|6.8,99.0|5.82,99.0|6.82,99.0|6.6
3,99.0|-0.74,8.35|3.32,99.0|0.67,99.0|1.62,1.8|1.45,8.16|4.16,-2.82|-0.45,6.21|-1.93,99.0|-2.3,1.84|-1.33,7.33|3.71,6.6|5.49,6.31|-0.37,8.11|6.67,-7.23|-3.36,-6.65|-5.1,1.17|2.24,-6.6|-3.34,-3.64|-0.25,-2.09|0.19,5.34|6.6,99.0|3.3,99.0|-0.58,99.0|-1.74,99.0|3.85,2.91|6.03,3.93|5.0,6.75|1.82,6.6|5.55,99.0|1.2,6.65|6.12,-6.12|3.09,99.0|0.54,7.57|5.47,6.21|4.35,6.65|3.7,99.0|-2.61,-8.3|0.47,7.18|5.72,2.82|3.29,...,0.0|5.27,-3.69|2.33,99.0|-0.24,99.0|-1.73,7.82|6.52,0.24|3.62,99.0|-1.66,7.28|2.93,-2.33|4.49,99.0|2.24,99.0|-1.14,99.0|3.15,99.0|3.22,99.0|-1.26,99.0|0.86,99.0|2.35,99.0|0.43,99.0|1.84,99.0|1.55,99.0|4.6,99.0|2.27,99.0|-1.81,99.0|3.64,99.0|-1.06,0.63|2.53,99.0|-4.99,99.0|1.09,-2.33|-1.12,99.0|0.98,99.0|-1.73,99.0|-1.94,99.0|-0.2,99.0|0.93,0.53|-3.76,99.0|-2.1,99.0|-1.84,99.0|-1.29,99.0|4.81,99.0|-0.95,99.0|5.5
4,8.5|2.37,4.61|3.06,-4.17|2.74,-5.39|1.99,1.36|2.81,1.6|5.16,7.04|0.73,4.61|-0.67,-0.44|1.04,5.73|2.41,8.25|4.88,6.84|5.22,-3.93|-1.21,7.23|5.65,-2.33|-2.15,-9.66|-3.26,2.72|0.99,-1.36|-0.59,2.57|2.01,4.51|1.16,8.2|6.31,6.12|3.96,8.3|2.12,-1.26|0.31,7.77|3.7,1.89|5.27,-1.17|6.16,5.68|4.11,8.45|5.54,4.61|2.68,8.06|5.76,-9.47|5.6,7.28|1.38,5.68|4.79,2.48|5.55,3.2|5.49,-1.26|-0.12,6.8|2.94,4.51|5.28,2.48|4.05,...,7.38|5.

#Finding best and worst rated jokes

In [128]:
#Taking mean of all the ratings for each joke to determine best to worst jokes
joke_rat=np.empty((data.shape[1],2), dtype=object)
for i in range (data.shape[1]):
  rat=[]
  for j in range (data.shape[0]):
    if (Actual_data[j][i]!=99):
      rat.append(Actual_data[j][i])
  joke_rat[i,0]="Joke:"+str(i)
  joke_rat[i,1]=np.mean(np.array(rat))

jk_ind=np.argsort(joke_rat[:,1])
jk_ind=jk_ind[::-1]
joke_rat=joke_rat[jk_ind,:]
df1 = pd.DataFrame(data=joke_rat)
print(df1)

          0        1
0   Joke:49  3.66508
1   Joke:88  3.57472
2   Joke:35   3.3125
3   Joke:26  3.18982
4   Joke:31  3.15599
..      ...      ...
95  Joke:12 -1.76288
96  Joke:56 -1.99066
97  Joke:43 -2.11189
98  Joke:15 -3.10482
99  Joke:57 -3.83388

[100 rows x 2 columns]
